In [1]:
import os
os.environ["JAX_PLATFORM_NAME"] = "cpu"
import netket as nk
import numpy as np
import matplotlib.pyplot as plt
import jax
import pandas as pd
import netket.nn as nknn
import flax.linen as nn
import jax.numpy as jnp
import matplotlib.pyplot as plt 
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
#Couplings J1 and J2
J = [1, 0.2]
L = 8

In [7]:
J[0]**2 + J[1]**2 

1.04

In [8]:
# Define custom graph
edge_colors = []
for i in range(L):
    edge_colors.append([i, (i+1)%L, 1])
    edge_colors.append([i, (i+2)%L, 2])
# Define the netket graph object
g = nk.graph.Graph(edges=edge_colors)

#Sigma^z*Sigma^z interactions
sigmaz = [[1, 0], [0, -1]]
mszsz = (np.kron(sigmaz, sigmaz))
exchange = np.asarray([[0, 0, 0, 0], [0, 0, 2, 0], [0, 2, 0, 0], [0, 0, 0, 0]])

bond_operator = [
    (J[0] * mszsz).tolist(),
    (J[1] * mszsz).tolist(),
    (-J[0] * exchange).tolist(),  
    (J[1] * exchange).tolist(),
]

bond_color = [1, 2, 1, 2]
hi = nk.hilbert.Spin(s=0.5, total_sz=0.0, N=g.n_nodes)

In [9]:
op = nk.operator.GraphOperator(hi, graph=g, bond_ops=bond_operator, bond_ops_colors=bond_color)

In [10]:
class FFNN(nn.Module):
    @nn.compact
    def __call__(self, x):
        x = nn.Dense(features=2*x.shape[-1], 
                     use_bias=True, 
                     param_dtype=np.complex128, 
                     kernel_init=nn.initializers.normal(stddev=0.01), 
                     bias_init=nn.initializers.normal(stddev=0.01)
                    )(x)
        x = nknn.log_cosh(x)
        x = jnp.sum(x, axis=-1)
        return x

model = FFNN()

In [11]:
sa = nk.sampler.MetropolisExchange(hilbert=hi, graph=g, d_max = 2)
vs = nk.vqs.MCState(sa, model, n_samples=1008)
opt = nk.optimizer.Sgd(learning_rate=0.01)
sr = nk.optimizer.SR(diag_shift=0.01)
gs = nk.VMC(hamiltonian=op, optimizer=opt, variational_state=vs, preconditioner=sr)

In [12]:
vs

MCState(
  hilbert = Spin(s=1/2, total_sz=0.0, N=8),
  sampler = MetropolisSampler(rule = ExchangeRule(# of clusters: 28), n_chains = 16, n_sweeps = 8, reset_chains = False, machine_power = 2, dtype = <class 'float'>),
  n_samples = 1008,
  n_discard_per_chain = 100,
  sampler_state = MetropolisSamplerState(rng state=[1181982813 3442651076]),
  n_parameters = 144)

In [13]:
vs_i_parameters = vs.parameters.copy()

In [14]:
vs_i_parameters['Dense_0']['bias']

Array([ 0.00195301-0.00013294j, -0.0187443 +0.00381496j,
       -0.00613122+0.00155427j, -0.00689251-0.00226213j,
       -0.00283473-0.00086334j, -0.00671474+0.00255856j,
        0.00016241+0.00089776j,  0.01240301-0.00952049j,
       -0.00600021+0.0012761j , -0.01128924-0.00370158j,
        0.00059864-0.00478035j,  0.00249781-0.00839709j,
        0.00920196+0.004698j  ,  0.00424503+0.00523527j,
        0.00944227+0.00756668j, -0.0084744 +0.00291845j],      dtype=complex128)

In [15]:
# Define a função de perda (loss function) para o treinamento
def loss(params, structure_factor):
    output = model.apply({'params': params}, structure_factor)
    return jnp.mean(output)
# Função de callback para salvar os parâmetros durante o treinamento
def save_params(step, params, energy):
    trained_params_list.append(params.copy())
    parameters_list.append(energy.state.parameters.copy())
    iii.append(1)
    return True

In [16]:
sf = []
sites = []
structure_factor = nk.operator.LocalOperator(hi, dtype=complex)
for i in range(0, L):
    for j in range(0, L):
        structure_factor += (nk.operator.spin.sigmaz(hi, i)*nk.operator.spin.sigmaz(hi, j))*((-1)**(i-j))/L

# Define uma lista para armazenar os parâmetros do modelo durante o treinamento
trained_params_list = []
parameters_list     = []
iii                 = []

In [18]:
gs.run(out='test', n_iter=100, obs={'Structure Factor': structure_factor}, callback=save_params)

100%|████████████████████████████████████████████████████████████████████████████| 100/100 [00:05<00:00, 16.87it/s, Energy=-13.40284-0.00015j ± 0.00020 [σ²=0.00004, R̂=1.0070]]


(JsonLog('test', mode=write, autoflush_cost=0.005)
   Runtime cost:
   	Log:    0.020716190338134766
   	Params: 0.0005695819854736328,)

In [ ]:
vs.parameters['Dense_0']['bias']

In [ ]:
len(parameters_list)

In [ ]:
parameters_list[-1]['Dense_0']['bias']

In [ ]:
def info(e):
    head   = list(e.keys())[0]
    body   = list(e[head].keys())
    bias   = e[head][body[0]]
    kernel = e[head][body[1]]
    return  head, body, list(bias), list(kernel)
def real(c):
    return float(np.real(c))  
def img(c):
    return float(np.imag(c))    
def r_i(c):
    return real(c),img(c)   

In [ ]:
head, body, bias_list,kernel_list = info(parameters_list[-1])

In [ ]:
len(bias_list)

In [ ]:
real_df = pd.DataFrame()
img_df  = pd.DataFrame()
for param in parameters_list:
    head, body, bias_list,kernel_list = info(param)
    real_v = [];img_v = []
    for bias in bias_list:
        nr, ni = r_i(bias); 
        real_v.append(nr)
        img_v.append(ni)   
    
    real_row_df = pd.DataFrame([real_v])
    img_row_df  = pd.DataFrame([img_v])
        
    real_df = pd.concat([real_df,real_row_df])
    img_df  = pd.concat([img_df,img_row_df])   

In [ ]:
real_df.shape

In [ ]:
img_df.shape

In [ ]:
real_df.head()

In [ ]:
real_df.columns

In [ ]:
real_df.insert(0, 'id', range(1, 1 + len(real_df)))

In [ ]:
real_df.columns

In [ ]:
img_df.insert(0, 'id', range(1, 1 + len(img_df)))

In [ ]:
img_df.head()

In [ ]:
real_df.plot('id',y=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15])
plt.show()


In [ ]:
img_df.plot('id',y=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15])
plt.show()

In [ ]:
real_df.plot('id',y=[0])
plt.show()

In [ ]:
img_df.plot('id',y=[0])
plt.show()

In [ ]:
len(kernel_list)

In [ ]:
len(kernel_list[0])

In [ ]:
(kernel_list[0][0])

In [ ]:
real_kernel_df = pd.DataFrame()
img_kernel_df  = pd.DataFrame()
for param in parameters_list:
    head, body, bias_list,kernel_list = info(param)
    real_v = [];img_v = []
    for ks in kernel_list:
        for k in ks:
            nr, ni = r_i(k); 
            real_v.append(nr)
            img_v.append(ni) 
    real_row_df = pd.DataFrame([real_v])
    img_row_df  = pd.DataFrame([img_v])
    
    real_kernel_df = pd.concat([real_kernel_df,real_row_df])
    img_kernel_df  = pd.concat([img_kernel_df,img_row_df])   

In [ ]:
real_kernel_df.insert(0, 'id', range(1, 1 + len(real_kernel_df)))
img_kernel_df.insert(0, 'id', range(1, 1 + len(img_kernel_df)))

In [ ]:
real_kernel_df.plot('id',y=[0])
plt.show()

In [ ]:
img_kernel_df.plot('id',y=[0])
plt.show()

In [ ]:
img_kernel_df.columns

In [ ]:
yt = []
for i in range(0,128): 
    if i % 8 ==0:
        yt.append(i)  
img_kernel_df.plot('id',y=yt)
plt.show()

In [ ]:
yt = []
for i in range(0,128): 
    yt.append(i)        

In [ ]:
img_kernel_df.plot('id',y=yt)
plt.show()

In [ ]:
real_kernel_df.plot('id',y=[7])
plt.show()

In [ ]:
yt = []
for i in range(0,128): 
    if i % 8 ==0:
        yt.append(i)  
real_kernel_df.plot('id',y=yt)
plt.show()

In [ ]:
yt = []
for i in range(0,128): 
    yt.append(i)  
real_kernel_df.plot('id',y=yt)
plt.show()